# Imports

In [5]:
!pip install python-dotenv

In [6]:
import requests
from kafka import KafkaProducer
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

In [10]:
# Charger les variables d'environnement depuis le fichier .env copié dans le conteneur
load_dotenv('/home/jovyan/.env')

# Configuration de l'API OpenSky
OPENSKY_URL = "https://opensky-network.org/api/states/all"
USERNAME = os.getenv('OPENSKY_USERNAME')
PASSWORD = os.getenv('OPENSKY_PASSWORD')

# Fonction pour envoyer les données OpenSky à Kafka

In [11]:

def send_opensky_to_kafka(topic, url, username, password):
    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from OpenSky
    response = requests.get(url, auth=(username, password))

    if response.status_code == 200:
        data = response.json()
        states = data.get("states", [])

        # Publish each state to Kafka
        for state in states:
            producer.send(topic, value=state)
            #print(f"Sent: {state}")

        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(states)} records.")
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [12]:
states = {
    "icao24": "icao24",
    "callsign": "callsign",
    "origin_country": "origin_country",
    "time_position": "time_position",
    "last_contact": "last_contact",
    "longitude": "longitude",
    "latitude": "latitude",
    "baro_altitude": "baro_altitude",
    "on_ground": "on_ground",
    "velocity": "velocity",
    "true_track": "true_track",
    "vertical_rate": "vertical_rate",
    "sensors": "sensors",
    "geo_altitude": "geo_altitude",
    "squawk": "squawk",
    "spi": "spi",
    "position_source": "position_source",
    "category": "category"
}

inital_date_str = "2025-01-25 16:00:00"

# Durée de récupération (minutes)
step = 60

# date initiale
initial_date_str = "2025-01-25 16:00:00"
# durée de récupération (minutes)
step = 60

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)


start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)




In [13]:
# Send OpenSky data to Kafka
import time
for i in range(10):
    print(f"{start}-{end}")
    send_opensky_to_kafka("opensky-flights", OPENSKY_URL, USERNAME, PASSWORD)
    time.sleep(5)
    initial_date = end_date
    end_date = initial_date + timedelta(minutes=step)
    start = initial_date.strftime(date_format)
    end = end_date.strftime(date_format)

2025-01-25 16:00:00-2025-01-25 17:00:00
Failed to fetch data: 403
2025-01-25 17:00:00-2025-01-25 18:00:00
Failed to fetch data: 403


KeyboardInterrupt: 